# 物体检测——测试视频脚本
## 使用说明
本脚本用于测试ModelArts模型管理中物体检测模型的视频测试

测试步骤：

（1）填写 input_video_path 参数，该参数指定了输入视频所在的路径，需要您提前上传视频到Notebook的某个路径；
 
（2）填写 video_start_time 参数和 video_end_time 参数，分别是指需要预测的视频的开始时间和结束时间，按照2位数字'时:分:秒'的格式进行填写

（3）到ModelArts模型管理中查看您要测试的模型，点击查看模型详情，找到model_id和AI引擎，将model_id复制填写到下面的test_model_id参数中，然后点击本页面顶部菜单栏的 “Kernel”-> “Change kernel”，选择与您的模型一致的引擎；


（4）点击本页面顶部菜单栏的 “Cell”-> “Run All”，本脚本将从输入视频路径加载视频，并将视频每帧画面的预测结果显示出来。

## 注意事项：
（1）本脚本的当前版本只在 Tensorflow 训练的模型上经过测试；

（2）如果您的模型预测结果标签为中文，则需要自行修改代码，以便正确地在图片上打印中文标签名

In [1]:
# 请填写如下四个参数
input_video_path = 'case_dev/mask_detect/datasets/videos/1.mp4'  # 输入视频所在的路径
video_start_time = '00:00:00'  # 设置需要处理的视频开始时间，按照2位数字'时:分:秒'的格式进行填写
video_end_time = '00:00:06'    # 设置需要处理的视频结束时间，按照2位数字'时:分:秒'的格式进行填写
test_model_id = '01c34d13-df31-4c30-ab3c-9ec37803dd86'    # 待测试模型的id

In [2]:
# 对路径做检查
import os
ROOT_DIR = '/home/ma-user/work'
input_video_path = os.path.join(ROOT_DIR, input_video_path)
if not os.path.exists(input_video_path):
    raise Exception("输入视频路径 input_video_path 不存在，请检查修改！")

### 本脚本将使用 ModelArts SDK 来部署测试模型，我们需要对sdk中的原有一个predictor.py进行修改才可以跑通本脚本，为了方便，可以直接使用已经修改好的脚本来覆盖原有的脚本

In [3]:
import shutil
shutil.copy('./predictor.py', '/home/ma-user/modelarts-sdk/modelarts/predictor.py')

'/home/ma-user/modelarts-sdk/modelarts/predictor.py'

In [4]:
# 将模型部署为本地服务
import cv2
import json
import codecs
import time, datetime
import numpy as np
import ipywidgets
from IPython.display import display
from PIL import Image, ImageDraw, ImageFont
from modelarts.session import Session
from modelarts.model import Model
from modelarts.config.model_config import ServiceConfig
session = Session()
Model.configure_tf_infer_environ(device_type='GPU')  # 如果不是使用 TF 训练的模型，则屏蔽这一行
model_instance = Model(session, model_id=test_model_id)
configs = [ServiceConfig(model_id=model_instance.model_id, weight="100", specification="local", instance_count=1)]
predictor_instance = model_instance.deploy_predictor(configs=configs)  # 部署为本地服务Predictor

Configuring tensorflow local inference environment ... 
Successfully configure tensorflow local inference environment
Service name is service-0210-053112
model_local_path is /home/ma-user/modelarts/model_location/model_id_01c34d13-df31-4c30-ab3c-9ec37803dd86/
Successfully download file obs-duqi/case_dev/mask_detect/model_snapshots_v2/model from OBS to local /home/ma-user/modelarts/model_location/model_id_01c34d13-df31-4c30-ab3c-9ec37803dd86/1/
local_service_port is 127.0.0.1:6060
Deploying the local service ...
Successfully deployed the local service.


In [5]:
# 定义视频读取函数
def read_video(input_video_path, video_start_time, video_end_time):
    cap = cv2.VideoCapture(input_video_path)  # 打开视频
    total_frame_num = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # 获取视频总帧数
    fps = cap.get(cv2.CAP_PROP_FPS)  # 视频帧率
    s_time_split = video_start_time.split(':')
    start_time = int(s_time_split[0]) * 3600 + int(s_time_split[1]) * 60 + int(s_time_split[2])
    e_time_split = video_end_time.split(':')
    end_time = int(e_time_split[0]) * 3600 + int(e_time_split[1]) * 60 + int(e_time_split[2])
    start_frame_id = int(start_time * fps)  # 设置需要处理的开始帧
    end_frame_id = int(end_time * fps)  # 设置需要处理的结束帧
    if end_frame_id > total_frame_num:
        end_frame_id = total_frame_num - 1
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame_id)  # 设置开始帧
    return cap, start_frame_id, end_frame_id

In [6]:
# 查看需要处理的视频
cap, start_frame_id, end_frame_id = read_video(input_video_path, video_start_time, video_end_time)
imgbox = ipywidgets.Image(format='jpg', height=int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)), width=int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)))  # 创建动态显示窗口
display(imgbox)  # 显示窗口内容
for frame_id in range(start_frame_id, end_frame_id):
    ret, frame = cap.read()
    cv2.putText(frame, 'id: ' + str(frame_id), (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)  # 画frame_id
    try:
        imgbox.value = cv2.imencode('.jpg', frame)[1].tobytes()
    except:
        continue
print('end')

Image(value=b'', format='jpg', height='368', width='640')

end


In [7]:
# 打开视频，获取每帧图，画框，显示
def detect(input_video_path, video_start_time, video_end_time):
    cap, start_frame_id, end_frame_id = read_video(input_video_path, video_start_time, video_end_time)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # 获取视频画面宽度
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # 获取视频画面高度
    imgbox = ipywidgets.Image(format='jpg', height=height, width=width)  # 创建动态显示窗口
    display(imgbox)  # 显示窗口内容

    for frame_id in range(start_frame_id + 1, end_frame_id + 1):
        ret, frame = cap.read()
        pil_frame = Image.fromarray(frame[:, :, ::-1])
        predict_result = predictor_instance.predict(data=pil_frame, data_type='images')     # 本地推理预测
        predict_result = json.loads(predict_result)
        classes = predict_result.get('detection_classes', None)
        boxes = predict_result.get('detection_boxes', None)
        scores = predict_result.get('detection_scores', None)          
        if classes is not None:
            for i in range(len(classes)):
                if float(scores[i]) < 0.95:
                    continue
                
                box = boxes[i]
                y1, x1, y2, x2 = [int(float(v)) for v in box]
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), thickness=2)
                
                # 注意！！！！，如果标签名为中文，则需要转成英文名再显示，因为cv2.puText只能显示英文
                text = classes[i] + '(%s)' % str(float(scores[i]) * 100)[:4] + '%'
                cv2.putText(frame, text, (x1, y1-3), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0))
        cv2.putText(frame, 'id: ' + str(frame_id), (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)  # 画frame_id
        imgbox.value = cv2.imencode('.jpg', frame)[1].tobytes()
    print('end')
detect(input_video_path, video_start_time, video_end_time)

Image(value=b'', format='jpg', height='368', width='640')

end
